# Import Libraries

In [61]:
import pandas as pd
import pyodbc
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Fetch Reviews From The Database

In [44]:
conn_str = (
    'DRIVER={SQL Server};'
    'SERVER=DESKTOP-PSV5L2B\SQLEXPRESS;'
    'DATABASE=DSS_DataWareHouse;'
    'Trusted_Connection=yes;'
)

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

query = 'SELECT * FROM Dim_Review'
query2 = 'SELECT * FROM Dim_Users'
query3 = 'SELECT * FROM Dim_Movies'
query4 = 'SELECT * FROM Fact_Reviews'

cursor.execute(query)
rows = cursor.fetchall()
reviews = pd.DataFrame.from_records(rows, columns=['ReviewID', 'Review', 'TokenCount'])

cursor.execute(query2)
rows = cursor.fetchall()
users = pd.DataFrame.from_records(rows, columns = ['UserID', 'UserName', 'TotalVotes'])

cursor.execute(query3)
rows = cursor.fetchall()
movies = pd.DataFrame.from_records(rows, columns=['MovieID', 'UserRating'])

cursor.execute(query4)
rows = cursor.fetchall()
fact = pd.DataFrame.from_records(rows, columns=['ReviewID', 'MovieID', 'UserID', 'Day', 'Month', 'Year'])


cursor.close()
conn.close()

<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Alireza HR\AppData\Local\Temp\ipykernel_17904\2510045194.py:3: SyntaxWarning: invalid escape sequence '\S'
  'SERVER=DESKTOP-PSV5L2B\SQLEXPRESS;'


In [45]:
temp = pd.merge(movies, fact, on="MovieID")
temp = temp.drop_duplicates()
temp

,MovieID,UserRating,ReviewID,UserID,Day,Month,Year
0,1,4,1,1,4,May,2019
1,1,4,2,2,6,May,2019
2,1,4,3,3,13,May,2019
3,1,4,4,4,3,August,2019
4,1,4,5,5,13,December,2021
...,...,...,...,...,...,...,...
47055,10,9,47037,40912,8,May,2018
47056,10,9,47038,40913,14,September,2018
47057,10,9,47039,40914,23,May,2018
47058,10,9,47040,40915,11,July,2018


# Association Rules

In [46]:
pivot_table = temp.pivot_table(index='UserID', columns='MovieID', values='UserRating', fill_value=0)

pivot_table_binary = pivot_table.applymap(lambda x: 1 if x > 0 else 0)

frequent_itemsets = apriori(pivot_table_binary, min_support=0.005, use_colnames=True)

rules = association_rules(frequent_itemsets, metric='lift', min_threshold=0.05)

rules = rules.sort_values(by='lift', ascending=False)
rules.head(10)


C:\Users\Alireza HR\AppData\Local\Temp\ipykernel_17904\2944486499.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table_binary = pivot_table.applymap(lambda x: 1 if x > 0 else 0)
c:\Users\Alireza HR\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
48,(10),"(1, 4)",0.045363,0.022779,0.005157,0.113685,4.990689,0.004124,1.102566,0.837624
45,"(1, 4)",(10),0.022779,0.045363,0.005157,0.226395,4.990689,0.004124,1.234010,0.818267
39,"(1, 4)",(3),0.022779,0.058660,0.005622,0.246781,4.207001,0.004285,1.249757,0.780071
42,(3),"(1, 4)",0.058660,0.022779,0.005622,0.095833,4.207001,0.004285,1.080797,0.809804
47,(1),"(10, 4)",0.229090,0.007577,0.005157,0.022511,2.971078,0.003421,1.015279,0.860570
46,"(10, 4)",(1),0.007577,0.229090,0.005157,0.680645,2.971078,0.003421,2.413960,0.668487
36,(8),(10),0.050838,0.045363,0.005939,0.116827,2.575354,0.003633,1.080917,0.644468
37,(10),(8),0.045363,0.050838,0.005939,0.130927,2.575354,0.003633,1.092154,0.640771
40,"(3, 4)",(1),0.010485,0.229090,0.005622,0.536131,2.340259,0.003219,1.661911,0.578765
41,(1),"(3, 4)",0.229090,0.010485,0.005622,0.024539,2.340259,0.003219,1.014407,0.742884


# Classification

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Download VADER lexicon
nltk.download('vader_lexicon')

# Load your dataset
dataset = reviews

[nltk_data] Downloading package vader_lexicon to C:\Users\Alireza
[nltk_data]     HR\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [48]:
# Initialize VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to classify sentiment
def classify_sentiment(text):
    sentiment_score = sid.polarity_scores(text)
    if sentiment_score['compound'] >= 0.05:
        return 'pos'
    elif sentiment_score['compound'] <= -0.05:
        return 'neg'
    else:
        return 'neu'

# Apply sentiment classification to the 'Review' column
dataset['Sentiment'] = dataset['Review'].apply(classify_sentiment)


In [49]:
dataset

,ReviewID,Review,TokenCount,Sentiment
0,1,"The writers got carried away, the directors ov...",1965,neg
1,2,Time travel is such a lazy way to write storie...,361,pos
2,3,Overrated and full of filler.\nContinuity? 350...,147,neg
3,4,"downbeat, overblown & so so long. Cuts all you...",995,pos
4,5,Not as good as infinity war but a great movie....,81,pos
...,...,...,...,...
47034,47037,It's a very good film..\nIn this movie is a lo...,55,pos
47035,47038,Too much.\nWhen Thor becomes Benny Hill. The s...,24,pos
47036,47039,Disney makes this movie ridiculous.\nI mean wh...,102,neg
47037,47040,WATCH MORE TAIKA.\nIf your takeaway from this ...,81,pos


# Clustering

In [56]:
# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Vectorize the 'Review' column
X = vectorizer.fit_transform(dataset['Review'])


In [57]:
# Initialize KMeans with 2 clusters (positive and negative)
kmeans = KMeans(n_clusters=3, random_state=40)

# Fit the model to the data
kmeans.fit(X)

# Add the cluster labels to the dataset
dataset['Cluster'] = kmeans.labels_
dataset

,ReviewID,Review,TokenCount,Sentiment,Cluster
0,1,"The writers got carried away, the directors ov...",1965,neg,1
1,2,Time travel is such a lazy way to write storie...,361,pos,1
2,3,Overrated and full of filler.\nContinuity? 350...,147,neg,1
3,4,"downbeat, overblown & so so long. Cuts all you...",995,pos,1
4,5,Not as good as infinity war but a great movie....,81,pos,1
...,...,...,...,...,...
47034,47037,It's a very good film..\nIn this movie is a lo...,55,pos,1
47035,47038,Too much.\nWhen Thor becomes Benny Hill. The s...,24,pos,1
47036,47039,Disney makes this movie ridiculous.\nI mean wh...,102,neg,1
47037,47040,WATCH MORE TAIKA.\nIf your takeaway from this ...,81,pos,1


In [62]:
# Calculate the silhouette score
silhouette_avg = silhouette_score(X, kmeans.labels_)

# Print the silhouette score
print(f'Silhouette Score: {silhouette_avg}')


Silhouette Score: 0.004181052376410673
